In [1]:
!git clone "https://github.com/convman/Multimodal.git"

Cloning into 'Multimodal'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 589 (delta 0), reused 0 (delta 0), pack-reused 575
Receiving objects: 100% (589/589), 1009.21 MiB | 30.75 MiB/s, done.
Resolving deltas: 100% (202/202), done.
Checking out files: 100% (346/346), done.


In [2]:
cd Multimodal/

/content/Multimodal


In [3]:
!chmod +x data/dataset_download.sh
!./data/dataset_download.sh

--2019-07-27 16:05:10--  http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/data/audio_test.h5
Resolving immortal.multicomp.cs.cmu.edu (immortal.multicomp.cs.cmu.edu)... 128.2.211.216
Connecting to immortal.multicomp.cs.cmu.edu (immortal.multicomp.cs.cmu.edu)|128.2.211.216|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57213024 (55M)
Saving to: ‘audio_test.h5’

audio_test.h5       100%[===================>]  54.56M  14.9MB/s    in 4.1s    

2019-07-27 16:05:15 (13.4 MB/s) - ‘audio_test.h5’ saved [57213024/57213024]

--2019-07-27 16:05:15--  http://immortal.multicomp.cs.cmu.edu/raw_datasets/old_processed_data/cmu-mosei/data/audio_train.h5
Resolving immortal.multicomp.cs.cmu.edu (immortal.multicomp.cs.cmu.edu)... 128.2.211.216
Connecting to immortal.multicomp.cs.cmu.edu (immortal.multicomp.cs.cmu.edu)|128.2.211.216|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 181035744 (173M)
Saving to: ‘audio_train.h5’

au

In [4]:
import pandas as pd
import numpy as np
import h5py
from sklearn.metrics import classification_report
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.utils.multiclass import unique_labels
import keras
import tensorflow as tf
from time import time
import scipy
from keras import layers
from google.colab import files	
from sklearn.utils import class_weight
from keras.models import load_model
from keras.models import Model,Sequential,Model
from keras.layers import *
from keras.callbacks import TensorBoard
from keras import callbacks
from keras.callbacks import *
%matplotlib inline

Using TensorFlow backend.


In [0]:
import pandas as pd
import numpy as np
import h5py

Test_labels_fear = np.array(pd.read_csv("mosi2uni_Test_labels_fear.csv",header=None))
Train_labels_fear = np.array(pd.read_csv("mosi2uni_Train_labels_fear.csv",header=None))

Test_labels_happy = np.array(pd.read_csv("mosi2uni_Test_labels_happy.csv",header=None))
Train_labels_happy = np.array(pd.read_csv("mosi2uni_Train_labels_happy.csv",header=None))

Test_labels_sad = np.array(pd.read_csv("mosi2uni_Test_labels_sad.csv",header=None))
Train_labels_sad = np.array(pd.read_csv("mosi2uni_Train_labels_sad.csv",header=None))

Test_labels_disgust = np.array(pd.read_csv("mosi2uni_Test_labels_disgust.csv",header=None))
Train_labels_disgust = np.array(pd.read_csv("mosi2uni_Train_labels_disgust.csv",header=None))

Test_labels_surprise = np.array(pd.read_csv("mosi2uni_Test_labels_surprise.csv",header=None))
Train_labels_surprise = np.array(pd.read_csv("mosi2uni_Train_labels_surprise.csv",header=None))

Test_labels_angry = np.array(pd.read_csv("mosi2uni_Test_labels_angry.csv",header=None))
Train_labels_angry = np.array(pd.read_csv("mosi2uni_Train_labels_angry.csv",header=None))

train_video = h5py.File("video_train.h5","r")
train_video = np.array(train_video.get('d1'))
test_video = h5py.File("video_test.h5","r")
test_video = np.array(test_video.get('d1'))

train_audio = h5py.File("audio_train.h5","r")
train_audio = np.array(train_audio.get('d1'))
test_audio = h5py.File("audio_test.h5","r")
test_audio = np.array(test_audio.get('d1'))

train_text = h5py.File("text_train_emb.h5","r")
train_text = np.array(train_text.get('d1'))
test_text = h5py.File("text_test_emb.h5","r")
test_text = np.array(test_text.get('d1'))

In [8]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced',np.unique(Train_labels_happy.squeeze()),Train_labels_happy.squeeze())

i_v = Input(shape=(20,35))
i_a = Input(shape=(20,74))
i_t = Input(shape=(20,300))

v = LSTM(64,return_sequences = True)(i_v)
v = LSTM(32,return_sequences = True)(v)
v = LSTM(32,return_sequences = False)(v)
v = Dense(32,activation='relu')(v)

a = LSTM(64,return_sequences = True)(i_a)
a = LSTM(32,return_sequences = True)(a)
a = LSTM(32,return_sequences = False)(a)
a = Dense(32,activation='relu')(a)

t = LSTM(128,return_sequences = True)(i_t)
t = LSTM(64,return_sequences = True)(t)
t = LSTM(32,return_sequences = False)(t)
t = Dense(32,activation='relu')(t)

merge = concatenate([v,a,t])
y = Dense(32,activation='relu')(merge)
y = Dense(1,activation='sigmoid')(y)

model = Model([i_v,i_a,i_t],y)
print(model.summary())
model.compile('adam','binary_crossentropy',metrics=['accuracy'])

es = EarlyStopping(monitor='val_acc',mode='max' ,patience=5, min_delta=0.0001,verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2,patience=2,min_lr=0.00001)
model.fit([train_video,train_audio,train_text],Train_labels_happy,512,epochs=100,validation_split=0.1,callbacks=[es,reduce_lr],
         class_weight=dict(enumerate(class_weight)))


y_pred = (model.predict([test_video,test_audio,test_text])>0.5).astype("float")
y_true = Test_labels_happy
cm = confusion_matrix(y_true,y_pred)

p = precision_score(y_true,y_pred)
a = accuracy_score(y_true,y_pred)
r = recall_score(y_true,y_pred)
f = f1_score(y_true,y_pred)
print(class_weight)
print(cm)
print("accuracy  :",a)
print("f1 score  :",f)
print("precision :",p)
print("recall    :",r)


TP = cm[1][1]
TN = cm[0][0]
P = Test_labels_happy.sum()
N = 4832-P
wa = ((TP*N/P) + TN)/(0.02*N)
print("WA        :",wa)
print(classification_report(y_true,y_pred))
model.save("LF_CW_happy.h5")

W0727 16:19:36.609092 139982898333568 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0727 16:19:36.639807 139982898333568 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0727 16:19:36.648356 139982898333568 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 20, 35)       0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 20, 74)       0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 20, 300)      0                                            
__________________________________________________________________________________________________
lstm_19 (LSTM)                  (None, 20, 64)       25600       input_7[0][0]                    
__________________________________________________________________________________________________
lstm_22 (L

W0727 16:19:43.602465 139982898333568 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 13761 samples, validate on 1529 samples
Epoch 1/100
13761/13761 [==============================] - 22s 2ms/step - loss: 0.6565 - acc: 0.6124 - val_loss: 0.6479 - val_acc: 0.6187
Epoch 2/100
13761/13761 [==============================] - 12s 841us/step - loss: 0.6068 - acc: 0.6703 - val_loss: 0.6315 - val_acc: 0.6266
Epoch 3/100
13761/13761 [==============================] - 12s 843us/step - loss: 0.5825 - acc: 0.6890 - val_loss: 0.6308 - val_acc: 0.6305
Epoch 4/100
13761/13761 [==============================] - 12s 846us/step - loss: 0.5715 - acc: 0.6941 - val_loss: 0.6387 - val_acc: 0.6377
Epoch 5/100
13761/13761 [==============================] - 12s 838us/step - loss: 0.5499 - acc: 0.7116 - val_loss: 0.6360 - val_acc: 0.6364
Epoch 6/100
13761/13761 [==============================] - 12s 837us/step - loss: 0.5325 - acc: 0.7254 - val_loss: 0.6489 - val_acc: 0.6422
Epoch 7/100
13761/13761 [==============================] - 12s 841us/step - loss: 0.5157 - acc: 0.7380 - val_loss

In [9]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced',np.unique(Train_labels_angry.squeeze()),Train_labels_angry.squeeze())

i_v = Input(shape=(20,35))
i_a = Input(shape=(20,74))
i_t = Input(shape=(20,300))

v = LSTM(64,return_sequences = True)(i_v)
v = LSTM(32,return_sequences = True)(v)
v = LSTM(32,return_sequences = False)(v)
v = Dense(32,activation='relu')(v)

a = LSTM(64,return_sequences = True)(i_a)
a = LSTM(32,return_sequences = True)(a)
a = LSTM(32,return_sequences = False)(a)
a = Dense(32,activation='relu')(a)

t = LSTM(128,return_sequences = True)(i_t)
t = LSTM(64,return_sequences = True)(t)
t = LSTM(32,return_sequences = False)(t)
t = Dense(32,activation='relu')(t)

merge = concatenate([v,a,t])
y = Dense(32,activation='relu')(merge)
y = Dense(1,activation='sigmoid')(y)

model = Model([i_v,i_a,i_t],y)
print(model.summary())
model.compile('adam','binary_crossentropy',metrics=['accuracy'])

es = EarlyStopping(monitor='val_acc',mode='max' ,patience=5, min_delta=0.0001,verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2,patience=2,min_lr=0.00001)
model.fit([train_video,train_audio,train_text],Train_labels_angry,512,epochs=100,validation_split=0.1,callbacks=[es,reduce_lr],
         class_weight=dict(enumerate(class_weight)))


y_pred = (model.predict([test_video,test_audio,test_text])>0.5).astype("float")
y_true = Test_labels_angry
cm = confusion_matrix(y_true,y_pred)

p = precision_score(y_true,y_pred)
a = accuracy_score(y_true,y_pred)
r = recall_score(y_true,y_pred)
f = f1_score(y_true,y_pred)
print(class_weight)
print(cm)
print("accuracy  :",a)
print("f1 score  :",f)
print("precision :",p)
print("recall    :",r)


TP = cm[1][1]
TN = cm[0][0]
P = Test_labels_angry.sum()
N = 4832-P
wa = ((TP*N/P) + TN)/(0.02*N)
print("WA        :",wa)
print(classification_report(y_true,y_pred))
model.save("LF_CW_angry.h5")

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 20, 35)       0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 20, 74)       0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 20, 300)      0                                            
__________________________________________________________________________________________________
lstm_28 (LSTM)                  (None, 20, 64)       25600       input_10[0][0]                   
__________________________________________________________________________________________________
lstm_31 (L

In [10]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced',np.unique(Train_labels_sad.squeeze()),Train_labels_sad.squeeze())

i_v = Input(shape=(20,35))
i_a = Input(shape=(20,74))
i_t = Input(shape=(20,300))

v = LSTM(64,return_sequences = True)(i_v)
v = LSTM(32,return_sequences = True)(v)
v = LSTM(32,return_sequences = False)(v)
v = Dense(32,activation='relu')(v)

a = LSTM(64,return_sequences = True)(i_a)
a = LSTM(32,return_sequences = True)(a)
a = LSTM(32,return_sequences = False)(a)
a = Dense(32,activation='relu')(a)

t = LSTM(128,return_sequences = True)(i_t)
t = LSTM(64,return_sequences = True)(t)
t = LSTM(32,return_sequences = False)(t)
t = Dense(32,activation='relu')(t)

merge = concatenate([v,a,t])
y = Dense(32,activation='relu')(merge)
y = Dense(1,activation='sigmoid')(y)

model = Model([i_v,i_a,i_t],y)
print(model.summary())
model.compile('adam','binary_crossentropy',metrics=['accuracy'])

es = EarlyStopping(monitor='val_acc',mode='max' ,patience=5, min_delta=0.0001,verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2,patience=2,min_lr=0.00001)
model.fit([train_video,train_audio,train_text],Train_labels_sad,512,epochs=100,validation_split=0.1,callbacks=[es,reduce_lr],
         class_weight=dict(enumerate(class_weight)))


y_pred = (model.predict([test_video,test_audio,test_text])>0.5).astype("float")
y_true = Test_labels_sad
cm = confusion_matrix(y_true,y_pred)

p = precision_score(y_true,y_pred)
a = accuracy_score(y_true,y_pred)
r = recall_score(y_true,y_pred)
f = f1_score(y_true,y_pred)
print(class_weight)
print(cm)
print("accuracy  :",a)
print("f1 score  :",f)
print("precision :",p)
print("recall    :",r)


TP = cm[1][1]
TN = cm[0][0]
P = Test_labels_sad.sum()
N = 4832-P
wa = ((TP*N/P) + TN)/(0.02*N)
print("WA        :",wa)
print(classification_report(y_true,y_pred))
model.save("LF_CW_sad.h5")

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 20, 35)       0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, 20, 74)       0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           (None, 20, 300)      0                                            
__________________________________________________________________________________________________
lstm_37 (LSTM)                  (None, 20, 64)       25600       input_13[0][0]                   
__________________________________________________________________________________________________
lstm_40 (L

In [14]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced',np.unique(Train_labels_fear.squeeze()),Train_labels_fear.squeeze())

i_v = Input(shape=(20,35))
i_a = Input(shape=(20,74))
i_t = Input(shape=(20,300))

v = LSTM(64,return_sequences = True)(i_v)
v = LSTM(32,return_sequences = True)(v)
v = LSTM(32,return_sequences = False)(v)
v = Dense(32,activation='relu')(v)

a = LSTM(64,return_sequences = True)(i_a)
a = LSTM(32,return_sequences = True)(a)
a = LSTM(32,return_sequences = False)(a)
a = Dense(32,activation='relu')(a)

t = LSTM(128,return_sequences = True)(i_t)
t = LSTM(64,return_sequences = True)(t)
t = LSTM(32,return_sequences = False)(t)
t = Dense(32,activation='relu')(t)

merge = concatenate([v,a,t])
y = Dense(32,activation='relu')(merge)
y = Dense(1,activation='sigmoid')(y)

model = Model([i_v,i_a,i_t],y)
print(model.summary())
model.compile('adam','binary_crossentropy',metrics=['accuracy'])

es = EarlyStopping(monitor='val_acc',mode='max' ,patience=5, min_delta=0.0001,verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2,patience=2,min_lr=0.00001)
model.fit([train_video,train_audio,train_text],Train_labels_fear,512,epochs=100,validation_split=0.1,callbacks=[es,reduce_lr],
         class_weight=dict(enumerate(class_weight)))


y_pred = (model.predict([test_video,test_audio,test_text])>0.5).astype("float")
y_true = Test_labels_fear
cm = confusion_matrix(y_true,y_pred)

p = precision_score(y_true,y_pred)
a = accuracy_score(y_true,y_pred)
r = recall_score(y_true,y_pred)
f = f1_score(y_true,y_pred)
print(class_weight)
print(cm)
print("accuracy  :",a)
print("f1 score  :",f)
print("precision :",p)
print("recall    :",r)


TP = cm[1][1]
TN = cm[0][0]
P = Test_labels_fear.sum()
N = 4832-P
wa = ((TP*N/P) + TN)/(0.02*N)
print("WA        :",wa)
print(classification_report(y_true,y_pred))
model.save("LF_CW_fear.h5")

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_25 (InputLayer)           (None, 20, 35)       0                                            
__________________________________________________________________________________________________
input_26 (InputLayer)           (None, 20, 74)       0                                            
__________________________________________________________________________________________________
input_27 (InputLayer)           (None, 20, 300)      0                                            
__________________________________________________________________________________________________
lstm_73 (LSTM)                  (None, 20, 64)       25600       input_25[0][0]                   
__________________________________________________________________________________________________
lstm_76 (L

In [12]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced',np.unique(Train_labels_surprise.squeeze()),Train_labels_surprise.squeeze())

i_v = Input(shape=(20,35))
i_a = Input(shape=(20,74))
i_t = Input(shape=(20,300))

v = LSTM(64,return_sequences = True)(i_v)
v = LSTM(32,return_sequences = True)(v)
v = LSTM(32,return_sequences = False)(v)
v = Dense(32,activation='relu')(v)

a = LSTM(64,return_sequences = True)(i_a)
a = LSTM(32,return_sequences = True)(a)
a = LSTM(32,return_sequences = False)(a)
a = Dense(32,activation='relu')(a)

t = LSTM(128,return_sequences = True)(i_t)
t = LSTM(64,return_sequences = True)(t)
t = LSTM(32,return_sequences = False)(t)
t = Dense(32,activation='relu')(t)

merge = concatenate([v,a,t])
y = Dense(32,activation='relu')(merge)
y = Dense(1,activation='sigmoid')(y)

model = Model([i_v,i_a,i_t],y)
print(model.summary())
model.compile('adam','binary_crossentropy',metrics=['accuracy'])

es = EarlyStopping(monitor='val_acc',mode='max' ,patience=5, min_delta=0.0001,verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2,patience=2,min_lr=0.00001)
model.fit([train_video,train_audio,train_text],Train_labels_surprise,512,epochs=100,validation_split=0.1,callbacks=[es,reduce_lr],
         class_weight=dict(enumerate(class_weight)))


y_pred = (model.predict([test_video,test_audio,test_text])>0.5).astype("float")
y_true = Test_labels_surprise
cm = confusion_matrix(y_true,y_pred)

p = precision_score(y_true,y_pred)
a = accuracy_score(y_true,y_pred)
r = recall_score(y_true,y_pred)
f = f1_score(y_true,y_pred)
print(class_weight)
print(cm)
print("accuracy  :",a)
print("f1 score  :",f)
print("precision :",p)
print("recall    :",r)


TP = cm[1][1]
TN = cm[0][0]
P = Test_labels_surprise.sum()
N = 4832-P
wa = ((TP*N/P) + TN)/(0.02*N)
print("WA        :",wa)
print(classification_report(y_true,y_pred))
model.save("LF_CW_surprise.h5")

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, 20, 35)       0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, 20, 74)       0                                            
__________________________________________________________________________________________________
input_21 (InputLayer)           (None, 20, 300)      0                                            
__________________________________________________________________________________________________
lstm_55 (LSTM)                  (None, 20, 64)       25600       input_19[0][0]                   
__________________________________________________________________________________________________
lstm_58 (L

In [13]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced',np.unique(Train_labels_disgust.squeeze()),Train_labels_disgust.squeeze())

i_v = Input(shape=(20,35))
i_a = Input(shape=(20,74))
i_t = Input(shape=(20,300))

v = LSTM(64,return_sequences = True)(i_v)
v = LSTM(32,return_sequences = True)(v)
v = LSTM(32,return_sequences = False)(v)
v = Dense(32,activation='relu')(v)

a = LSTM(64,return_sequences = True)(i_a)
a = LSTM(32,return_sequences = True)(a)
a = LSTM(32,return_sequences = False)(a)
a = Dense(32,activation='relu')(a)

t = LSTM(128,return_sequences = True)(i_t)
t = LSTM(64,return_sequences = True)(t)
t = LSTM(32,return_sequences = False)(t)
t = Dense(32,activation='relu')(t)

merge = concatenate([v,a,t])
y = Dense(32,activation='relu')(merge)
y = Dense(1,activation='sigmoid')(y)

model = Model([i_v,i_a,i_t],y)
print(model.summary())
model.compile('adam','binary_crossentropy',metrics=['accuracy'])

es = EarlyStopping(monitor='val_acc',mode='max' ,patience=5, min_delta=0.0001,verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2,patience=2,min_lr=0.00001)
model.fit([train_video,train_audio,train_text],Train_labels_disgust,512,epochs=100,validation_split=0.1,callbacks=[es,reduce_lr],
         class_weight=dict(enumerate(class_weight)))


y_pred = (model.predict([test_video,test_audio,test_text])>0.5).astype("float")
y_true = Test_labels_disgust
cm = confusion_matrix(y_true,y_pred)

p = precision_score(y_true,y_pred)
a = accuracy_score(y_true,y_pred)
r = recall_score(y_true,y_pred)
f = f1_score(y_true,y_pred)
print(class_weight)
print(cm)
print("accuracy  :",a)
print("f1 score  :",f)
print("precision :",p)
print("recall    :",r)


TP = cm[1][1]
TN = cm[0][0]
P = Test_labels_disgust.sum()
N = 4832-P
wa = ((TP*N/P) + TN)/(0.02*N)
print("WA        :",wa)
print(classification_report(y_true,y_pred))
model.save("LF_CW_disgust.h5")

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           (None, 20, 35)       0                                            
__________________________________________________________________________________________________
input_23 (InputLayer)           (None, 20, 74)       0                                            
__________________________________________________________________________________________________
input_24 (InputLayer)           (None, 20, 300)      0                                            
__________________________________________________________________________________________________
lstm_64 (LSTM)                  (None, 20, 64)       25600       input_22[0][0]                   
__________________________________________________________________________________________________
lstm_67 (L

In [0]:
files.download("LF_CW_happy.h5")
files.download("LF_CW_angry.h5")
files.download("LF_CW_sad.h5")
files.download("LF_CW_fear.h5")
files.download("LF_CW_surprise.h5")
files.download("LF_CW_disgust.h5")
